Импорт модулей

In [1]:
import cv2 as cv
from mtcnn import MTCNN
import tensorflow as tf
import face_recognition

print(cv.__version__)

Using TensorFlow backend.


4.2.0


Описание констант

In [2]:
RED = (0, 0, 255)

Отключение отладочной информации

In [3]:
tf.get_logger().setLevel('ERROR')

# Система распознавания лиц

Открывание камеры для захвата видео

In [4]:
cap = cv.VideoCapture(0)

if not cap.isOpened():
    print('Не удаётся открыть камеру')

Создание экземпляра свёрточной нейронной сети для обнаружения лиц

In [5]:
detector = MTCNN()

Подготовка набора данных лиц

In [6]:
known_faces = []

labels = []
labels.append('mikhailov')

for label in labels:
    image = cv.imread('known_faces/{}.jpg'.format(label))
    if image is None:
        break
    
    faces = detector.detect_faces(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    if not len(faces) == 1:
        break
    
    x1, y1 = faces[0]['box'][0], faces[0]['box'][1]
    x2, y2 = faces[0]['box'][0] + faces[0]['box'][2], faces[0]['box'][1] + faces[0]['box'][3]
    known_faces.append({'encoding': face_recognition.face_encodings(image[y1:y2, x1:x2])[0], 'label': label})

print(known_faces)

[{'encoding': array([-0.06119749,  0.06527939,  0.01028452,  0.00078343, -0.08590816,
       -0.00620787, -0.02763153, -0.03398338,  0.07434861, -0.1012755 ,
        0.29058209, -0.04339862, -0.26764905, -0.13614407,  0.0676738 ,
        0.10001744, -0.10210194, -0.02750669, -0.09622415,  0.05079712,
        0.11064393,  0.05344667,  0.07290477,  0.09125496, -0.16081746,
       -0.3938967 , -0.07471377, -0.07663848,  0.00697561, -0.14085314,
       -0.1184682 ,  0.09515718, -0.10877727, -0.02771313, -0.01339164,
        0.07218896, -0.08048238, -0.05791171,  0.20593958,  0.03780343,
       -0.21111108, -0.00727309, -0.07932518,  0.28276291,  0.2377969 ,
       -0.00618066,  0.0333436 , -0.01753489,  0.06977507, -0.3201797 ,
        0.0493774 ,  0.14129178,  0.05567065, -0.00493846,  0.06752561,
       -0.11232223, -0.01667364,  0.11067906, -0.26405954,  0.0277695 ,
       -0.01067162, -0.14994444, -0.01129279, -0.05861608,  0.15194947,
        0.11558464, -0.1182877 , -0.16035683,  0.0

Отображение видео в окне

In [7]:
while True:
    ret, frame = cap.read()
    if not ret:
        print('Не удаётся получить кадр')
        break
    
    faces = detector.detect_faces(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    for face in faces:
        x1, y1, x2, y2 = face['box'][0], face['box'][1], face['box'][0] + face['box'][2], face['box'][1] + face['box'][3]
        cv.rectangle(frame, (x1, y1), (x2, y2), RED, 3)
        
        label = 'unknown'
        try:
            encoding = face_recognition.face_encodings(frame[y1:y2, x1:x2])[0]
            
            for known_face in known_faces:
                if face_recognition.compare_faces([known_face['encoding']], encoding)[0] == True:
                    label = known_face['label']
        except:
            pass
        
        cv.putText(frame, label, (x1, y1), cv.FONT_HERSHEY_SIMPLEX, 1, RED)
    
    cv.imshow('Facial recognition system', frame)
    
    if cv.waitKey(1) == ord('q'):
        break

cv.destroyAllWindows()

Освобождение ресурсов

In [8]:
cap.release()